# Finished

In [2]:
import copy
import commons
from einops import rearrange, einsum
import math
import numpy as np
import torch
from torch import nn
from torch.nn import Conv1d, ConvTranspose1d, Conv2d
from torch.nn import functional as F
from torch.nn.utils import weight_norm, remove_weight_norm
# The following codes is SynthesizerTrn in VITS, let analyze it.
# Feel free to skip this class SynthesizerTrn.
# """
# class SynthesizerTrn(nn.Module):

#   def __init__(self, 
#     n_vocab,
#     spec_channels,
#     segment_size,
#     inter_channels,
#     hidden_channels,
#     filter_channels,
#     n_heads,
#     n_layers,
#     kernel_size,
#     p_dropout,
#     resblock, 
#     resblock_kernel_sizes, 
#     resblock_dilation_sizes, 
#     upsample_rates, 
#     upsample_initial_channel, 
#     upsample_kernel_sizes,
#     n_speakers=0,
#     gin_channels=0,
#     use_sdp=True,
#     **kwargs):

#     super().__init__()
#     self.n_vocab = n_vocab
#     self.spec_channels = spec_channels
#     self.inter_channels = inter_channels
#     self.hidden_channels = hidden_channels
#     self.filter_channels = filter_channels
#     self.n_heads = n_heads
#     self.n_layers = n_layers
#     self.kernel_size = kernel_size
#     self.p_dropout = p_dropout
#     self.resblock = resblock
#     self.resblock_kernel_sizes = resblock_kernel_sizes
#     self.resblock_dilation_sizes = resblock_dilation_sizes
#     self.upsample_rates = upsample_rates
#     self.upsample_initial_channel = upsample_initial_channel
#     self.upsample_kernel_sizes = upsample_kernel_sizes
#     self.segment_size = segment_size
#     self.n_speakers = n_speakers
#     self.gin_channels = gin_channels
    
#     self.use_sdp = use_sdp

#     self.enc_p = TextEncoder(n_vocab,
#         inter_channels,
#         hidden_channels,
#         filter_channels,
#         n_heads,
#         n_layers,
#         kernel_size,
#         p_dropout)
#     self.dec = Generator(inter_channels, resblock, resblock_kernel_sizes, resblock_dilation_sizes, upsample_rates, upsample_initial_channel, upsample_kernel_sizes, gin_channels=gin_channels)
#     self.enc_q = PosteriorEncoder(spec_channels, inter_channels, hidden_channels, 5, 1, 16, gin_channels=gin_channels)
#     self.flow = ResidualCouplingBlock(inter_channels, hidden_channels, 5, 1, 4, gin_channels=gin_channels)

#     if use_sdp:
#       self.dp = StochasticDurationPredictor(hidden_channels, 192, 3, 0.5, 4, gin_channels=gin_channels)
#     else:
#       self.dp = DurationPredictor(hidden_channels, 256, 3, 0.5, gin_channels=gin_channels)

#     if n_speakers > 1:
#       self.emb_g = nn.Embedding(n_speakers, gin_channels)

#   def forward(self, x, x_lengths, y, y_lengths, sid=None):

#     x, m_p, logs_p, x_mask = self.enc_p(x, x_lengths)
#     if self.n_speakers > 0:
#       g = self.emb_g(sid).unsqueeze(-1) # [b, h, 1]
#     else:
#       g = None

#     z, m_q, logs_q, y_mask = self.enc_q(y, y_lengths, g=g)
#     z_p = self.flow(z, y_mask, g=g)

#     with torch.no_grad():
#       # negative cross-entropy 
#       s_p_sq_r = torch.exp(-2 * logs_p) # [b, d, t]
#       neg_cent1 = torch.sum(-0.5 * math.log(2 * math.pi) - logs_p, [1], keepdim=True) # [b, 1, t_s]
#       neg_cent2 = torch.matmul(-0.5 * (z_p ** 2).transpose(1, 2), s_p_sq_r) # [b, t_t, d] x [b, d, t_s] = [b, t_t, t_s]
#       neg_cent3 = torch.matmul(z_p.transpose(1, 2), (m_p * s_p_sq_r)) # [b, t_t, d] x [b, d, t_s] = [b, t_t, t_s]
#       neg_cent4 = torch.sum(-0.5 * (m_p ** 2) * s_p_sq_r, [1], keepdim=True) # [b, 1, t_s]
#       neg_cent = neg_cent1 + neg_cent2 + neg_cent3 + neg_cent4

#       attn_mask = torch.unsqueeze(x_mask, 2) * torch.unsqueeze(y_mask, -1)
#       attn = monotonic_align.maximum_path(neg_cent, attn_mask.squeeze(1)).unsqueeze(1).detach()

#     w = attn.sum(2)
#     if self.use_sdp:
#       l_length = self.dp(x, x_mask, w, g=g)
#       l_length = l_length / torch.sum(x_mask)
#     else:
#       logw_ = torch.log(w + 1e-6) * x_mask
#       logw = self.dp(x, x_mask, g=g)
#       l_length = torch.sum((logw - logw_)**2, [1,2]) / torch.sum(x_mask) # for averaging 

#     # expand prior
#     m_p = torch.matmul(attn.squeeze(1), m_p.transpose(1, 2)).transpose(1, 2)
#     logs_p = torch.matmul(attn.squeeze(1), logs_p.transpose(1, 2)).transpose(1, 2)

#     z_slice, ids_slice = commons.rand_slice_segments(z, y_lengths, self.segment_size)
#     o = self.dec(z_slice, g=g)
#     return o, l_length, attn, ids_slice, x_mask, y_mask, (z, z_p, m_p, logs_p, m_q, logs_q)

#   def infer(self, x, x_lengths, sid=None, noise_scale=1, length_scale=1, noise_scale_w=1., max_len=None):
#     x, m_p, logs_p, x_mask = self.enc_p(x, x_lengths)
#     if self.n_speakers > 0:
#       g = self.emb_g(sid).unsqueeze(-1) # [b, h, 1]
#     else:
#       g = None

#     if self.use_sdp:
#       logw = self.dp(x, x_mask, g=g, reverse=True, noise_scale=noise_scale_w)
#     else:
#       logw = self.dp(x, x_mask, g=g)
#     w = torch.exp(logw) * x_mask * length_scale
#     w_ceil = torch.ceil(w)
#     y_lengths = torch.clamp_min(torch.sum(w_ceil, [1, 2]), 1).long()
#     y_mask = torch.unsqueeze(commons.sequence_mask(y_lengths, None), 1).to(x_mask.dtype)
#     attn_mask = torch.unsqueeze(x_mask, 2) * torch.unsqueeze(y_mask, -1)
#     attn = commons.generate_path(w_ceil, attn_mask)

#     m_p = torch.matmul(attn.squeeze(1), m_p.transpose(1, 2)).transpose(1, 2) # [b, t', t], [b, t, d] -> [b, d, t']
#     logs_p = torch.matmul(attn.squeeze(1), logs_p.transpose(1, 2)).transpose(1, 2) # [b, t', t], [b, t, d] -> [b, d, t']

#     z_p = m_p + torch.randn_like(m_p) * torch.exp(logs_p) * noise_scale
#     z = self.flow(z_p, y_mask, g=g, reverse=True)
#     o = self.dec((z * y_mask)[:,:,:max_len], g=g)
#     return o, attn, y_mask, (z, z_p, m_p, logs_p)

#   def voice_conversion(self, y, y_lengths, sid_src, sid_tgt):
#     assert self.n_speakers > 0, "n_speakers have to be larger than 0."
#     g_src = self.emb_g(sid_src).unsqueeze(-1)
#     g_tgt = self.emb_g(sid_tgt).unsqueeze(-1)
#     z, m_q, logs_q, y_mask = self.enc_q(y, y_lengths, g=g_src)
#     z_p = self.flow(z, y_mask, g=g_src)
#     z_hat = self.flow(z_p, y_mask, g=g_tgt, reverse=True)
#     o_hat = self.dec(z_hat * y_mask, g=g_tgt)
#     return o_hat, y_mask, (z, z_p, z_hat)
# """

c:\Users\linka\anaconda3\envs\pytorch\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\linka\anaconda3\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\linka\anaconda3\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
# Define variables in the Class
n_vocab = 178 # len(symbols)
spec_channels = 1024 // 2 + 1
segment_size = 8192 // 256
inter_channels = 192
hidden_channels = 192
filter_channels = 768
n_heads = 2
n_layers = 6
kernel_size = 3
p_dropout = 0.1
resblock = "1"
resblock_kernel_sizes = [3, 7, 11]
resblock_dilation_sizes = [[1, 3, 5], [1, 3, 5], [1, 3, 5]]
upsample_rates = [8, 8, 2, 2]
upsample_initial_channel = 512
upsample_kernel_sizes = [16, 16, 4, 4]
n_speakers = 0
gin_channels = 0
use_sdp = False

In [4]:
"""
self.enc_p = TextEncoder(n_vocab,
        inter_channels,
        hidden_channels,
        filter_channels,
        n_heads,
        n_layers,
        kernel_size,
        p_dropout)
self.dec = Generator(inter_channels, resblock, resblock_kernel_sizes, resblock_dilation_sizes, upsample_rates, upsample_initial_channel, upsample_kernel_sizes, gin_channels=gin_channels)
self.enc_q = PosteriorEncoder(spec_channels, inter_channels, hidden_channels, 5, 1, 16, gin_channels=gin_channels)
self.flow = ResidualCouplingBlock(inter_channels, hidden_channels, 5, 1, 4, gin_channels=gin_channels)

self.dp = DurationPredictor(hidden_channels, 256, 3, 0.5, gin_channels=gin_channels)

# we ignore emb_g = nn.Embedding(n_speakers) because it is used for multi-speaker TTS, which is not our case.
"""
# We need to define TextEncoder, Generator, PosteriorEncoder, ResidualCouplingBlock, DurationPredictor.


'\nself.enc_p = TextEncoder(n_vocab,\n        inter_channels,\n        hidden_channels,\n        filter_channels,\n        n_heads,\n        n_layers,\n        kernel_size,\n        p_dropout)\nself.dec = Generator(inter_channels, resblock, resblock_kernel_sizes, resblock_dilation_sizes, upsample_rates, upsample_initial_channel, upsample_kernel_sizes, gin_channels=gin_channels)\nself.enc_q = PosteriorEncoder(spec_channels, inter_channels, hidden_channels, 5, 1, 16, gin_channels=gin_channels)\nself.flow = ResidualCouplingBlock(inter_channels, hidden_channels, 5, 1, 4, gin_channels=gin_channels)\n\nself.dp = DurationPredictor(hidden_channels, 256, 3, 0.5, gin_channels=gin_channels)\n\n# we ignore emb_g = nn.Embedding(n_speakers) because it is used for multi-speaker TTS, which is not our case.\n'

![model](./static/fig_1a.png)

enc_p convert text into μ and σ (variational inference). Use mean and standard deviation to give variability to the model.

enc_q(posterior encoder)convert linear spectrogram into z

dec is decoder between z and waveform

flow convert z into f_{θ}(z), it is invertible, which means we can use a invert flow to convert f_{θ}(z') into z'

dp is DurationPredictor that predict the length of each phoneme

In [5]:
## Let's implement Text Encoder First
##################### This submodel is really hard #####################
## You may skip it and just consider it as a MultiheadAttention with Relative Positional Encoding##
class RelativeMultiHeadAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.heads_share = True # Default is True, we only demonstrate True here.
        self.windows_size = 4 # Default is 4
        self.k_channels = hidden_channels // n_heads # multiheads attention channels is hidden_channels / n_heads

        # attention convolution layers
        self.conv_q = nn.Conv1d(hidden_channels, hidden_channels, 1)
        self.conv_k = nn.Conv1d(hidden_channels, hidden_channels, 1)
        self.conv_v = nn.Conv1d(hidden_channels, hidden_channels, 1)
        self.conv_o = nn.Conv1d(hidden_channels, hidden_channels, 1)

        self.drop = nn.Dropout(p_dropout)

        rel_stddev = self.k_channels ** -0.5
        self.emb_rel_k = nn.Parameter(torch.randn(1, self.windows_size * 2 + 1, self.k_channels) * rel_stddev) # First dimension is 1, because we want it share heads
        self.emb_rel_v = nn.Parameter(torch.randn(1, self.windows_size * 2 + 1, self.k_channels) * rel_stddev)

        # initialize conv weights
        nn.init.xavier_uniform_(self.conv_q.weight)
        nn.init.xavier_uniform_(self.conv_k.weight)
        nn.init.xavier_uniform_(self.conv_v.weight)
    
    def forward(self, x, c, attn_mask=None):
        q = self.conv_q(x)
        k = self.conv_k(c)
        v = self.conv_v(c)
        
        x, self.attn = self.attention(q, k, v, mask=attn_mask)

        x = self.conv_o(x)
        return x

    def attention(self, query, key, value, mask=None):
        # reshape [batch, hidden_channel, length] -> [batch, number of heads(2), length, hidden_channel / number of heads(d_k)]
        b, d, t_s, t_t = (*key.size(), query.size(2)) # t_s must be equal to t_t, t_s is the length of key, t_t is the length of query
        # Since it is self attention, so query = key = value, t_s = t_t
        query = rearrange(query, 'b (h d) t -> b h t d', h=2)
        key = rearrange(key, 'b (h d) t -> b h t d', h=2)
        value = rearrange(value, 'b (h d) t -> b h t d', h=2)

        # scores = torch.matmul(query / math.sqrt(self.k_channels), key.transpose(-2, -1)) # Query * Key
        scores = einsum(query, key, 'b h t1 d, b h t2 d -> b h t1 t2') # Query * Key^T
        key_relative_embeddings = self._get_relative_embeddings(self.emb_rel_k, t_s)
        rel_logits = self._matmul_with_relative_keys(query / math.sqrt(self.k_channels), key_relative_embeddings) #Query * Key embedding
        scores_local = self._relative_position_to_absolute_position(rel_logits)

        scores = scores + scores_local
        scores = scores.masked_fill(mask == 0, -1e4)

        p_attn = F.softmax(scores, dim=-1) # [b, n_h, t_t, t_s]
        p_attn = self.drop(p_attn)

        output = torch.matmul(p_attn, value) # attn * value
        relative_weights = self._absolute_position_to_relative_position(p_attn)
        value_relative_embeddings = self._get_relative_embeddings(self.emb_rel_v, t_s)
        output = output + self._matmul_with_relative_values(relative_weights, value_relative_embeddings) # attn * value embedding
        output = output.transpose(2, 3).contiguous().view(b, d, t_t) # [b, n_h, t_t, d_k] -> [b, d, t_t]
        return output, p_attn

    def _matmul_with_relative_values(self, x, y):
        """
        x: [b, h, l, m]
        y: [h or 1, m, d]
        ret: [b, h, l, d]
        b batch
        h heads
        l length
        m 2*length-1
        d hidden / heads

        """
        # ret = torch.matmul(x, y.unsqueeze(0))
        ret = einsum(x, y, "b h l m, s m d -> b h l d")
        return ret

    def _matmul_with_relative_keys(self, x, y):
        """
        x: [b, h, l, d]
        y: [h or 1, m, d]
        ret: [b, h, l, m]
        """
        # ret = torch.matmul(x, y.unsqueeze(0).transpose(-2, -1))
        ret = einsum(x, y, "b h l d, s m d -> b h l m")
        return ret

    def _get_relative_embeddings(self, relative_embeddings, length):
        max_relative_position = 2 * self.windows_size + 1
        # Pad first before slice to avoid using cond ops.
        pad_length = max(length - (self.windows_size + 1), 0)
        slice_start_position = max((self.windows_size + 1) - length, 0)
        slice_end_position = slice_start_position + 2 * length - 1
        if pad_length > 0:
            padded_relative_embeddings = F.pad(
                relative_embeddings,
                commons.convert_pad_shape([[0, 0], [pad_length, pad_length], [0, 0]])) # Just add zero to the second dimenision
                # For instance, if the original shape is [1, 9, 158], after padding, it will be [1, 9 + 2 * pad_length(add 0 to both pad_length before and after), 158]
        else:
            padded_relative_embeddings = relative_embeddings
        used_relative_embeddings = padded_relative_embeddings[:,slice_start_position:slice_end_position]
        return used_relative_embeddings
    

    def _relative_position_to_absolute_position(self, x): # Simply consider this method as a change of shape without losing any information
        """
        x: [b, h, l, 2*l-1]
        ret: [b, h, l, l]
        """
        batch, heads, length, _ = x.size()
        # Concat columns of pad to shift from relative to absolute indexing.
        x = F.pad(x, commons.convert_pad_shape([[0,0],[0,0],[0,0],[0,1]]))

        # Concat extra elements so to add up to shape (len+1, 2*len-1).
        x_flat = x.view([batch, heads, length * 2 * length])
        x_flat = F.pad(x_flat, commons.convert_pad_shape([[0,0],[0,0],[0,length-1]]))

        # Reshape and slice out the padded elements.
        x_final = x_flat.view([batch, heads, length+1, 2*length-1])[:, :, :length, length-1:]
        # print(x_final[0][0][0:3])
        return x_final

    def _absolute_position_to_relative_position(self, x): # Simply consider this method as a change of shape without losing any information
        """
        x: [b, h, l, l]
        ret: [b, h, l, 2*l-1]
        """
        batch, heads, length, _ = x.size()
        # padd along column
        x = F.pad(x, commons.convert_pad_shape([[0, 0], [0, 0], [0, 0], [0, length-1]]))
        x_flat = x.view([batch, heads, length**2 + length*(length -1)])
        # add 0's in the beginning that will skew the elements after reshape
        x_flat = F.pad(x_flat, commons.convert_pad_shape([[0, 0], [0, 0], [length, 0]]))
        x_final = x_flat.view([batch, heads, length, 2*length])[:,:,:,1:]
        return x_final

In [6]:
RMHA = RelativeMultiHeadAttention()
print(RMHA)
print("Number of Parameters:", sum(p.numel() for p in RMHA.parameters() if p.requires_grad))

RelativeMultiHeadAttention(
  (conv_q): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
  (conv_k): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
  (conv_v): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
  (conv_o): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
  (drop): Dropout(p=0.1, inplace=False)
)
Number of Parameters: 149952


In [7]:
class LayerNorm(nn.Module):
  def __init__(self, channels, eps=1e-5):
    super().__init__()
    self.channels = channels
    self.eps = eps

    self.gamma = nn.Parameter(torch.ones(channels))
    self.beta = nn.Parameter(torch.zeros(channels))

  def forward(self, x):
    # x = x.transpose(1, -1)
    x = rearrange(x, 'b h l -> b l h')
    x = F.layer_norm(x, (self.channels,), self.gamma, self.beta, self.eps)
    return rearrange(x, 'b l h -> b h l')

In [8]:
class FFN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv_1 = nn.Conv1d(hidden_channels, filter_channels, kernel_size)
    self.conv_2 = nn.Conv1d(filter_channels, hidden_channels, kernel_size)
    self.drop = nn.Dropout(p_dropout)

  def forward(self, x, x_mask):
    x = self.conv_1(self.padding(x * x_mask))
    x = torch.relu(x)
    x = self.drop(x)
    x = self.conv_2(self.padding(x * x_mask))
    return x * x_mask

  def padding(self, x):
    pad_l = (kernel_size - 1) // 2
    pad_r = kernel_size // 2
    padding = [[0, 0], [0, 0], [pad_l, pad_r]]
    x = F.pad(x, commons.convert_pad_shape(padding))
    return x

In [9]:
FFNNet = FFN()
print(FFNNet)
print("Number of Parameters:", sum(p.numel() for p in FFNNet.parameters() if p.requires_grad))

FFN(
  (conv_1): Conv1d(192, 768, kernel_size=(3,), stride=(1,))
  (conv_2): Conv1d(768, 192, kernel_size=(3,), stride=(1,))
  (drop): Dropout(p=0.1, inplace=False)
)
Number of Parameters: 885696


In [10]:
class Encoder(nn.Module):
  def __init__(self):
    super().__init__()
    
    self.drop = nn.Dropout(p_dropout)
    self.attn_layers = nn.ModuleList()
    self.norm_layers_1 = nn.ModuleList()
    self.ffn_layers = nn.ModuleList()
    self.norm_layers_2 = nn.ModuleList()
    for i in range(n_layers):
      self.attn_layers.append(RelativeMultiHeadAttention())
      self.norm_layers_1.append(LayerNorm(hidden_channels))
      self.ffn_layers.append(FFN())
      self.norm_layers_2.append(LayerNorm(hidden_channels))

  def forward(self, x, x_mask):
    attn_mask = x_mask.unsqueeze(2) * x_mask.unsqueeze(-1)
    x = x * x_mask
    for i in range(n_layers):
      y = self.attn_layers[i](x, x, attn_mask) # x(1, 192, 158), y(1, 192, 158)
      y = self.drop(y)
      x = self.norm_layers_1[i](x + y)

      y = self.ffn_layers[i](x, x_mask)
      y = self.drop(y)
      x = self.norm_layers_2[i](x + y)
    x = x * x_mask
    return x

In [11]:
EncoderNet = Encoder()
print(EncoderNet)
print("Number of Parameters:", sum(p.numel() for p in EncoderNet.parameters() if p.requires_grad))

Encoder(
  (drop): Dropout(p=0.1, inplace=False)
  (attn_layers): ModuleList(
    (0-5): 6 x RelativeMultiHeadAttention(
      (conv_q): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
      (conv_k): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
      (conv_v): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
      (conv_o): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
      (drop): Dropout(p=0.1, inplace=False)
    )
  )
  (norm_layers_1): ModuleList(
    (0-5): 6 x LayerNorm()
  )
  (ffn_layers): ModuleList(
    (0-5): 6 x FFN(
      (conv_1): Conv1d(192, 768, kernel_size=(3,), stride=(1,))
      (conv_2): Conv1d(768, 192, kernel_size=(3,), stride=(1,))
      (drop): Dropout(p=0.1, inplace=False)
    )
  )
  (norm_layers_2): ModuleList(
    (0-5): 6 x LayerNorm()
  )
)
Number of Parameters: 6218496


In [12]:
class TextEncoder(nn.Module):
  def __init__(self,
      n_vocab,
      out_channels,
      hidden_channels,
      filter_channels,
      n_heads,
      n_layers,
      kernel_size,
      p_dropout):
    super().__init__()
    self.n_vocab = n_vocab
    self.out_channels = out_channels
    self.hidden_channels = hidden_channels
    self.filter_channels = filter_channels
    self.n_heads = n_heads
    self.n_layers = n_layers
    self.kernel_size = kernel_size
    self.p_dropout = p_dropout
    # The parameters in configs.

    self.emb = nn.Embedding(n_vocab, hidden_channels)
    nn.init.normal_(self.emb.weight, 0.0, hidden_channels**-0.5)

    self.encoder = Encoder()
    self.proj= nn.Conv1d(hidden_channels, out_channels * 2, 1)

  def forward(self, x, x_lengths):
    x = self.emb(x) * math.sqrt(self.hidden_channels) # [b, l, h]
    x = rearrange(x, 'b l h -> b h l') # [b, h, l]
    x_mask = torch.unsqueeze(commons.sequence_mask(x_lengths, x.size(2)), 1).to(x.dtype)

    x = self.encoder(x * x_mask, x_mask)

    stats = self.proj(x) * x_mask # Statistics, mean and log standard deviation.

    m, logs = torch.split(stats, self.out_channels, dim=1)
    return x, m, logs, x_mask

In [13]:
text_encoder = TextEncoder(n_vocab,
        inter_channels,
        hidden_channels,
        filter_channels,
        n_heads,
        n_layers,
        kernel_size,
        p_dropout)

In [14]:
# num parameters of textencoder
print("Number of Parameters:", sum(p.numel() for p in text_encoder.parameters() if p.requires_grad)) 

Number of Parameters: 6326784


In [15]:
textinput = torch.randint(0, 100, (1, 158))

In [16]:
print(textinput)

tensor([[46, 95, 43, 85, 80, 28, 94, 88, 85, 63, 82,  9, 91, 87, 71, 67, 74, 57,
         11,  4, 95, 11, 67,  7, 61, 66,  1, 92, 89, 23, 97, 46, 46, 97, 12, 84,
         15, 46, 79, 60, 48, 58, 83,  6, 20, 44,  8, 97,  0, 29, 26, 19, 88, 17,
         62,  2, 71,  8, 18, 34, 19, 49, 51,  4, 51, 84, 10, 49, 25, 61, 81, 60,
         60, 19, 64, 66, 97, 95,  0, 46, 71, 21, 12, 52, 86,  3, 15, 50, 71, 78,
         94, 75, 20, 53, 47, 34, 93, 51, 56, 59, 58, 76, 81, 91, 19,  8, 70, 90,
         90, 49, 56,  6, 55, 79, 66, 20, 11, 99, 20,  0, 10, 28, 14, 57, 66,  1,
         43, 80, 68, 84, 10, 85, 38, 38,  0, 11, 46, 74, 66,  4, 26, 32, 85, 76,
         42, 31, 96, 23, 27, 14, 62, 49, 95, 15, 57, 39, 54, 16]])


In [17]:
res_TextEncoder = text_encoder(textinput, torch.tensor([158, 150]))
print("text embedding:\n",res_TextEncoder[0])
print("text embedding mean:\n",res_TextEncoder[1])
print("text embedding stadard deviation:\n", res_TextEncoder[2])
print("text embedding mask:\n", res_TextEncoder[3])

text embedding:
 tensor([[[-0.4576,  0.5749,  1.3347,  ...,  1.6136,  1.0378, -0.2792],
         [ 1.9822, -0.6755,  1.2458,  ...,  0.3858, -0.3078,  0.1002],
         [ 1.0432,  0.5308,  0.4790,  ...,  0.0633,  0.9655,  0.6004],
         ...,
         [-0.4707, -0.0253, -0.5916,  ...,  0.0631,  1.7575, -1.5492],
         [ 1.1636,  0.5467,  2.0369,  ..., -0.2976, -0.3359,  0.1192],
         [-0.3192, -0.1926,  0.7349,  ..., -0.1634, -0.5856, -0.7213]],

        [[ 1.0182,  1.0698,  2.3990,  ...,  0.0000, -0.0000, -0.0000],
         [ 1.2327, -1.0745, -0.2581,  ..., -0.0000, -0.0000, -0.0000],
         [ 0.7931,  0.5947, -0.9162,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.8073,  0.6346, -0.8061,  ..., -0.0000, -0.0000, -0.0000],
         [-0.0926,  0.2835,  0.3877,  ..., -0.0000, -0.0000, -0.0000],
         [-0.0428, -0.6163,  1.8590,  ..., -0.0000, -0.0000, -0.0000]]],
       grad_fn=<MulBackward0>)
text embedding mean:
 tensor([[[ 0.1308, -0.7709,  0.0299,  ...,  0.

In [18]:
####################### From the output above, you may know what does TextEncoding DO #######################

![model](./static/TextEncoder.png)

The circled part is Implemented Above

In [19]:
#################### Implementing Posterior Encoder####################
class WN(torch.nn.Module):
  def __init__(self, n_layers, p_dropout=0, dilation_rate=1): # We only leave n_layers and p_dropout here because 
    # both PosteriorEncoder and Flow use WN, with different layers and dropout.
    super(WN, self).__init__()
    assert(kernel_size % 2 == 1)
    self.n_layers = n_layers
    self.p_dropout = p_dropout

    self.in_layers = torch.nn.ModuleList()
    self.res_skip_layers = torch.nn.ModuleList()
    self.drop = nn.Dropout(p_dropout)

    for i in range(n_layers):
      dilation = dilation_rate ** i
      padding = int((kernel_size * dilation - dilation) / 2)
      in_layer = torch.nn.Conv1d(hidden_channels, 2*hidden_channels, kernel_size,
                                 dilation=dilation, padding=padding)
      in_layer = torch.nn.utils.weight_norm(in_layer, name='weight')
      self.in_layers.append(in_layer)

      # last one is not necessary
      if i < n_layers - 1:
        res_skip_channels = 2 * hidden_channels
      else:
        res_skip_channels = hidden_channels

      res_skip_layer = torch.nn.Conv1d(hidden_channels, res_skip_channels, 1)
      res_skip_layer = torch.nn.utils.weight_norm(res_skip_layer, name='weight')
      self.res_skip_layers.append(res_skip_layer)

  def forward(self, x, x_mask, g=None, **kwargs):
    output = torch.zeros_like(x)
    n_channels_tensor = torch.IntTensor([hidden_channels])

    for i in range(self.n_layers):
      x_in = self.in_layers[i](x)
      g_l = torch.zeros_like(x_in)

      acts = commons.fused_add_tanh_sigmoid_multiply(
          x_in,
          g_l,
          n_channels_tensor)
      acts = self.drop(acts)

      res_skip_acts = self.res_skip_layers[i](acts)
      if i < self.n_layers - 1:
        res_acts = res_skip_acts[:,:hidden_channels,:]
        x = (x + res_acts) * x_mask
        output = output + res_skip_acts[:,hidden_channels:,:]
      else:
        output = output + res_skip_acts
    return output * x_mask

In [20]:
WeightNormNet = WN(4)
print(WeightNormNet)
print(sum(p.numel() for p in WeightNormNet.parameters() if p.requires_grad))

WN(
  (in_layers): ModuleList(
    (0-3): 4 x Conv1d(192, 384, kernel_size=(3,), stride=(1,), padding=(1,))
  )
  (res_skip_layers): ModuleList(
    (0-2): 3 x Conv1d(192, 384, kernel_size=(1,), stride=(1,))
    (3): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
  )
  (drop): Dropout(p=0, inplace=False)
)
1148544


In [21]:
class PosteriorEncoder(nn.Module):
  def __init__(self,
      kernel_size=5,
      dilation_rate=1,
      n_layers=16,):
    super().__init__()

    self.pre = nn.Conv1d(spec_channels, hidden_channels, 1)
    self.enc = WN(n_layers)
    self.proj = nn.Conv1d(hidden_channels, inter_channels * 2, 1)

  def forward(self, x, x_lengths):
    x_mask = torch.unsqueeze(commons.sequence_mask(x_lengths, x.size(2)), 1).to(x.dtype)
    x = self.pre(x) * x_mask
    x = self.enc(x, x_mask)
    stats = self.proj(x) * x_mask
    m, logs = torch.split(stats, inter_channels, dim=1)
    z = (m + torch.randn_like(m) * torch.exp(logs)) * x_mask
    return z, m, logs, x_mask

In [22]:
PosteriorEncoderNet = PosteriorEncoder()
print(PosteriorEncoderNet)
print("Number of Parameters:",sum(p.numel() for p in PosteriorEncoderNet.parameters() if p.requires_grad))

PosteriorEncoder(
  (pre): Conv1d(513, 192, kernel_size=(1,), stride=(1,))
  (enc): WN(
    (in_layers): ModuleList(
      (0-15): 16 x Conv1d(192, 384, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (res_skip_layers): ModuleList(
      (0-14): 15 x Conv1d(192, 384, kernel_size=(1,), stride=(1,))
      (15): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
    )
    (drop): Dropout(p=0, inplace=False)
  )
  (proj): Conv1d(192, 384, kernel_size=(1,), stride=(1,))
)
Number of Parameters: 4878720


![model](./static/PosteriorEncoder.png)

The circled part is implemented above

In [23]:
spec = torch.randn(2, 513, 1000) # [b, c, l]

In [24]:
res_PosteriorEncoder = PosteriorEncoderNet(spec, torch.tensor([1000, 900])) # (z, μ, log σ, mask)

In [25]:
print("The tail of res_PosteriorEncoder[0][0] is not 0, because it is the longest one in the batch:\n",
       res_PosteriorEncoder[0][0])
print("The tail of res_PosteriorEncoder[0][1] is 0, because it is shorter with a mask of 45000 in the batch:\n",
       res_PosteriorEncoder[0][1])

The tail of res_PosteriorEncoder[0][0] is not 0, because it is the longest one in the batch:
 tensor([[ 1.6924, -0.9372, -0.3818,  ..., -0.1471,  0.5407, -0.5183],
        [-0.5498,  0.1947,  0.5965,  ...,  0.8779, -0.4923, -0.8225],
        [-1.5736,  0.7595,  1.1712,  ...,  0.0348,  1.3333, -1.1421],
        ...,
        [ 2.2577,  1.1967,  0.5119,  ..., -0.7482,  1.7190,  0.8182],
        [-0.3651, -0.9726,  0.2499,  ..., -1.7469,  0.9346,  0.0098],
        [ 1.1019,  1.6298, -1.3040,  ..., -2.0117,  0.5795,  0.6038]],
       grad_fn=<SelectBackward0>)
The tail of res_PosteriorEncoder[0][1] is 0, because it is shorter with a mask of 45000 in the batch:
 tensor([[ 1.3601, -0.3390, -0.2801,  ...,  0.0000, -0.0000,  0.0000],
        [-0.0205,  1.1932,  1.6059,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1169, -0.0648,  0.1934,  ..., -0.0000, -0.0000, -0.0000],
        ...,
        [ 0.2366, -0.7224, -1.4271,  ..., -0.0000,  0.0000, -0.0000],
        [-0.4124, -0.4694,  1.9994,  ..., 

In [26]:
################## Flow ##################
"""
If you do not have a background knowledge of Flow,
For a better understanding of Flow here, please look at this articles.
https://jaketae.github.io/study/glowtts/
"""
class ResidualCouplingLayer(nn.Module):
  def __init__(self,
      kernel_size = 5,
      dilation_rate = 1,
      n_layers = 4,
      p_dropout=0):
    super().__init__()
    self.kernel_size = kernel_size
    self.dilation_rate = dilation_rate
    self.n_layers = n_layers
    self.half_channels = inter_channels // 2

    self.pre = nn.Conv1d(self.half_channels, hidden_channels, 1)
    self.enc = WN(n_layers, p_dropout=p_dropout)
    self.post = nn.Conv1d(hidden_channels, self.half_channels, 1)
    self.post.weight.data.zero_()
    self.post.bias.data.zero_()

  def forward(self, x, x_mask, reverse=False):
    x0, x1 = torch.split(x, [self.half_channels]*2, 1)
    h = self.pre(x0) * x_mask
    h = self.enc(h, x_mask)
    stats = self.post(h) * x_mask
    m = stats
    logs = torch.zeros_like(m)

    if not reverse:
      x1 = m + x1 * torch.exp(logs) * x_mask
      x = torch.cat([x0, x1], 1)
      logdet = torch.sum(logs, [1,2])
      return x, logdet
    else:
      x1 = (x1 - m) * torch.exp(-logs) * x_mask
      x = torch.cat([x0, x1], 1)
      return x

In [27]:
ResidualCouplingLayerNet = ResidualCouplingLayer()
print(ResidualCouplingLayerNet)
print("Number of Parameters:",sum(p.numel() for p in ResidualCouplingLayerNet.parameters() if p.requires_grad))

ResidualCouplingLayer(
  (pre): Conv1d(96, 192, kernel_size=(1,), stride=(1,))
  (enc): WN(
    (in_layers): ModuleList(
      (0-3): 4 x Conv1d(192, 384, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (res_skip_layers): ModuleList(
      (0-2): 3 x Conv1d(192, 384, kernel_size=(1,), stride=(1,))
      (3): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
    )
    (drop): Dropout(p=0, inplace=False)
  )
  (post): Conv1d(192, 96, kernel_size=(1,), stride=(1,))
)
Number of Parameters: 1185696


In [28]:
class Flip(nn.Module):
  def forward(self, x, *args, reverse=False, **kwargs):
    x = torch.flip(x, [1])
    if not reverse:
      logdet = torch.zeros(x.size(0)).to(dtype=x.dtype, device=x.device) # log of determinant of Jacobian
      return x, logdet # If you do not understand logdet, look at the article above.
    else:
      return x

In [29]:
flip = Flip()
x = torch.randn(1, 10, 20)
print("Channel 0 before flip: \n",x[0][0])
print("Channel 9 before flip: \n", x[0][9])
print("Channel 0 after flip: \n", flip(x)[0][0][0])
print("Channel 9 after flip: \n", flip(x)[0][0][9])

Channel 0 before flip: 
 tensor([-0.0209, -0.5983, -0.6825, -0.7250, -1.6613, -0.4718,  0.0447, -0.4561,
         0.2376, -1.1804, -0.2487,  0.1643,  0.8962,  1.4512,  0.2645, -0.8177,
         0.4190, -0.7258,  0.0643,  0.1317])
Channel 9 before flip: 
 tensor([ 0.0230,  0.5165,  1.4632, -1.9552, -0.0457, -0.4816,  1.5400,  1.3102,
        -1.0153, -0.2790,  0.1535,  0.8963,  0.4098, -0.0438, -1.7034,  1.3477,
        -0.1509,  0.4305,  1.0784, -1.7767])
Channel 0 after flip: 
 tensor([ 0.0230,  0.5165,  1.4632, -1.9552, -0.0457, -0.4816,  1.5400,  1.3102,
        -1.0153, -0.2790,  0.1535,  0.8963,  0.4098, -0.0438, -1.7034,  1.3477,
        -0.1509,  0.4305,  1.0784, -1.7767])
Channel 9 after flip: 
 tensor([-0.0209, -0.5983, -0.6825, -0.7250, -1.6613, -0.4718,  0.0447, -0.4561,
         0.2376, -1.1804, -0.2487,  0.1643,  0.8962,  1.4512,  0.2645, -0.8177,
         0.4190, -0.7258,  0.0643,  0.1317])


In [30]:
class ResidualCouplingBlock(nn.Module):
  def __init__(self,
      kernel_size=5,
      dilation_rate=1,
      n_layers=4,
      n_flows=4):
    super().__init__()
    self.flows = nn.ModuleList()
    for i in range(n_flows):
      self.flows.append(ResidualCouplingLayer(kernel_size=kernel_size, dilation_rate=dilation_rate, n_layers=n_layers))
      self.flows.append(Flip())

  def forward(self, x, x_mask, reverse=False):
    if not reverse:
      for flow in self.flows:
        x, _ = flow(x, x_mask, reverse=reverse)
    else:
      for flow in reversed(self.flows):
        x = flow(x, x_mask, reverse=reverse)
    return x


In [31]:
Flow = ResidualCouplingBlock()
print(Flow)
print("Number of Parameters:",sum(p.numel() for p in Flow.parameters() if p.requires_grad))

ResidualCouplingBlock(
  (flows): ModuleList(
    (0): ResidualCouplingLayer(
      (pre): Conv1d(96, 192, kernel_size=(1,), stride=(1,))
      (enc): WN(
        (in_layers): ModuleList(
          (0-3): 4 x Conv1d(192, 384, kernel_size=(3,), stride=(1,), padding=(1,))
        )
        (res_skip_layers): ModuleList(
          (0-2): 3 x Conv1d(192, 384, kernel_size=(1,), stride=(1,))
          (3): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
        )
        (drop): Dropout(p=0, inplace=False)
      )
      (post): Conv1d(192, 96, kernel_size=(1,), stride=(1,))
    )
    (1): Flip()
    (2): ResidualCouplingLayer(
      (pre): Conv1d(96, 192, kernel_size=(1,), stride=(1,))
      (enc): WN(
        (in_layers): ModuleList(
          (0-3): 4 x Conv1d(192, 384, kernel_size=(3,), stride=(1,), padding=(1,))
        )
        (res_skip_layers): ModuleList(
          (0-2): 3 x Conv1d(192, 384, kernel_size=(1,), stride=(1,))
          (3): Conv1d(192, 192, kernel_size=(1,), stride=(1,

In [32]:
Flow_res = Flow(res_PosteriorEncoder[0], res_PosteriorEncoder[3])

In [33]:
RevFlow_res = Flow(Flow_res, res_PosteriorEncoder[3], reverse=True)

In [34]:
if (RevFlow_res - res_PosteriorEncoder[0]).sum() < 1e-6:
  print("The Flow model is reversible") # Flow should be reversible.

The Flow model is reversible


![model](./static/Flow.png)

The circled part is implemented above

In [35]:
############# Decoder #############
def get_padding(kernel_size, dilation=1): # keep dimension
  return int((kernel_size*dilation - dilation)/2)

def init_weights(m, mean=0.0, std=0.01):
  classname = m.__class__.__name__
  if classname.find("Conv") != -1:
    m.weight.data.normal_(mean, std)

class ResBlock1(torch.nn.Module): # Dilated Convolution
    def __init__(self, channels, kernel_size=3, dilation=(1, 3, 5)):
        super(ResBlock1, self).__init__()
        self.convs1 = nn.ModuleList([
            weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=dilation[0],
                               padding=get_padding(kernel_size, dilation[0]))),
            weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=dilation[1],
                               padding=get_padding(kernel_size, dilation[1]))),
            weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=dilation[2],
                               padding=get_padding(kernel_size, dilation[2])))
        ])
        self.convs1.apply(init_weights)

        self.convs2 = nn.ModuleList([
            weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=1,
                               padding=get_padding(kernel_size, 1))),
            weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=1,
                               padding=get_padding(kernel_size, 1))),
            weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=1,
                               padding=get_padding(kernel_size, 1)))
        ])
        self.convs2.apply(init_weights)

    def forward(self, x, x_mask=None):
        for c1, c2 in zip(self.convs1, self.convs2):
            xt = F.leaky_relu(x, 0.1)
            if x_mask is not None:
                xt = xt * x_mask
            xt = c1(xt)
            xt = F.leaky_relu(xt, 0.1)
            if x_mask is not None:
                xt = xt * x_mask
            xt = c2(xt)
            x = xt + x
        if x_mask is not None:
            x = x * x_mask
        return x

In [36]:
resblock1 = ResBlock1(192)
print(resblock1)
print("Number of Parameters:",sum(p.numel() for p in resblock1.parameters() if p.requires_grad))

ResBlock1(
  (convs1): ModuleList(
    (0): Conv1d(192, 192, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): Conv1d(192, 192, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
    (2): Conv1d(192, 192, kernel_size=(3,), stride=(1,), padding=(5,), dilation=(5,))
  )
  (convs2): ModuleList(
    (0-2): 3 x Conv1d(192, 192, kernel_size=(3,), stride=(1,), padding=(1,))
  )
)
Number of Parameters: 665856


In [37]:
class Generator(torch.nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.num_kernels = len(resblock_kernel_sizes)
        self.num_upsamples = len(upsample_rates)
        self.conv_pre = Conv1d(inter_channels, upsample_initial_channel, 7, 1, padding=3)
        resblock = ResBlock1

        self.ups = nn.ModuleList()
        for i, (u, k) in enumerate(zip(upsample_rates, upsample_kernel_sizes)):
            self.ups.append(weight_norm(
                ConvTranspose1d(upsample_initial_channel//(2**i), upsample_initial_channel//(2**(i+1)),
                                k, u, padding=(k-u)//2)))

        self.resblocks = nn.ModuleList()
        for i in range(len(self.ups)):
            ch = upsample_initial_channel//(2**(i+1))
            for j, (k, d) in enumerate(zip(resblock_kernel_sizes, resblock_dilation_sizes)):
                self.resblocks.append(resblock(ch, k, d))

        self.conv_post = Conv1d(ch, 1, 7, 1, padding=3, bias=False)
        self.ups.apply(init_weights)

        if gin_channels != 0:
            self.cond = nn.Conv1d(gin_channels, upsample_initial_channel, 1)

    def forward(self, x):
        x = self.conv_pre(x)

        for i in range(self.num_upsamples):
            x = F.leaky_relu(x, 0.1)
            x = self.ups[i](x)
            xs = None
            for j in range(self.num_kernels):
                if xs is None:
                    xs = self.resblocks[i*self.num_kernels+j](x)
                else:
                    xs += self.resblocks[i*self.num_kernels+j](x)
            x = xs / self.num_kernels
        x = F.leaky_relu(x)
        x = self.conv_post(x)
        x = torch.tanh(x)

        return x

In [38]:
decoder = Generator()
print(decoder)
print("Number of Parameters:",sum(p.numel() for p in decoder.parameters() if p.requires_grad))

Generator(
  (conv_pre): Conv1d(192, 512, kernel_size=(7,), stride=(1,), padding=(3,))
  (ups): ModuleList(
    (0): ConvTranspose1d(512, 256, kernel_size=(16,), stride=(8,), padding=(4,))
    (1): ConvTranspose1d(256, 128, kernel_size=(16,), stride=(8,), padding=(4,))
    (2): ConvTranspose1d(128, 64, kernel_size=(4,), stride=(2,), padding=(1,))
    (3): ConvTranspose1d(64, 32, kernel_size=(4,), stride=(2,), padding=(1,))
  )
  (resblocks): ModuleList(
    (0): ResBlock1(
      (convs1): ModuleList(
        (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
        (2): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(5,), dilation=(5,))
      )
      (convs2): ModuleList(
        (0-2): 3 x Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      )
    )
    (1): ResBlock1(
      (convs1): ModuleList(
        (0): Conv1d(256, 256, kernel_size=(7,), stride=(1,),

In [39]:
"""
After decoder, we can get the waveform. It is used in inference.
"""
res_Decoder_PosteriorEncoder = decoder(res_PosteriorEncoder[0]) # Waveform

In [40]:
print(res_Decoder_PosteriorEncoder.shape)

torch.Size([2, 1, 256000])


![model](./static/Decoder.png)

The circled part is implemented above

### Monotonic Alignment Search
Recall that VITS deals with two input modalities during training:

A string of text
Its corresponding mel-spectrogram
The mel-spectrogram is decoded through the flow-based decoder. Similarly, the text is fed to a text encoder network, which outputs mean (μ) and standard deviation (σ) for each token of text.

For example, given the string of text ["h", "e", "l", "l", "o"], we would have a total of five mean and standard deviation vectors corresponding to each letter. We can denote them as:

Mean vectors: μ1, μ2, ..., μ5
Standard deviation vectors: σ1, σ2, ..., σ5
Let's assume in this example that the corresponding mel-spectrogram spans a total of 100 frames. The output of the flow decoder would also be 100 vectors, denoted as z1, z2, ..., z100.

Reference: https://jaketae.github.io/study/glowtts/

In [41]:
###### In the description above, we have the mean, logσ, and z. ######
mean = res_TextEncoder[1][0:2] # Our batch size is 2, we select the first one.
log_sigma = res_TextEncoder[2][0:2]
z = Flow_res[0:2]
print("shape of μ", mean.shape)
print("shape of σ",log_sigma.shape)
print("shape of z(Spectrogram)",z.shape)

shape of μ torch.Size([2, 192, 158])
shape of σ torch.Size([2, 192, 158])
shape of z(Spectrogram) torch.Size([2, 192, 1000])


In [42]:
###### We will have a Likelihood Scores matrix, shape is (1, 158, 1000) ######
s_p_sq_r = torch.exp(-2 * log_sigma) # [b, d, t_t]
neg_cent1 = torch.sum(-0.5 * math.log(2 * math.pi) - log_sigma, [1], keepdim=True) # [b, 1, t_t]
neg_cent2 = torch.matmul(-0.5 * (z ** 2).transpose(1, 2), s_p_sq_r) # [b, t_t, d] x [b, d, t_s] = [b, t_s, t_t]
neg_cent3 = torch.matmul(z.transpose(1, 2), (mean * s_p_sq_r)) # [b, t_t, d] x [b, d, t_s] = [b, t_s, t_t]
neg_cent4 = torch.sum(-0.5 * (mean ** 2) * s_p_sq_r, [1], keepdim=True) # [b, 1, t_t]
neg_cent = neg_cent1 + neg_cent2 + neg_cent3 + neg_cent4 # [b, t_s, t_t]

#### Don't understand why we calculate it like this?

What is s_p_sq_r? the $σ^-2$

What is neg_cent1? $-0.5*log(2*pi) - log(σ)$

What is neg_cent2? $-0.5*z^2*σ^-2 = -\frac{z^2}{2σ^2}$

What is neg_cent3? $z * μ * σ^-2 = \frac{z*μ}{σ^-2}$

What is neg_cent4? $-0.5*μ^2*σ^-2 = -\frac{μ^2}{2σ^2}$

What is neg_cent? $-0.5*log(2*pi) - log(σ) + (-\frac{z^2}{2σ^2}) + \frac{z*μ}{σ^-2} + (-\frac{μ^2}{2σ^2})$

$                 =-0.5*log(2*pi) - log(σ) - \frac{x^2-2xμ+μ^2}{2σ^2}$

$                 =-0.5*log(2*pi) - log(σ) - \frac{(x-μ)^2}{2σ^2}$

$                 =log(p(x))=log(\frac{1}{\sqrt{2\pi}\sigma}exp(-\frac{(x-\mu)^2}{2\sigma^2}))$

That is, log of likelihood

In [43]:
########### Now, we have a matrix of shape of (batch, length of spectrogram, length of text) ###########
## Next step, we will find a road to get the best alignment between spectrogram and text. ##
## For instance, we have length 3 for text and length 5 for spectrogram
## Then, our matrix will be (1, 5, 3), and we will find the best alignment between them.
## The alignment matrix is looked like this
## 1 0 0 
## 0 1 0
## 0 0 1
## 0 0 1
## 0 0 1
## The requirement is start from the left-top corner, and end at the right-bottom corner
## each step can only move to the right-down or down.
## the column is the text, we can not skip any text
## the row is the spectrogram, we can not skip any spectrogram or use any frame of spectrogram twice.
## The best alignment is the path that can result highest likelihood.

In [44]:
res_TextEncoder[3].shape

torch.Size([2, 1, 158])

In [45]:
res_PosteriorEncoder[3].shape

torch.Size([2, 1, 1000])

In [46]:
######### Don't forget the mask, we can not use the padding part. #########
attn_mask = (torch.unsqueeze(res_TextEncoder[3], 2) * torch.unsqueeze(res_PosteriorEncoder[3], -1)).squeeze(1)

In [47]:
print("attn_mask shape:", attn_mask.shape)
print("Second attn_mask:\n", attn_mask[1])
print("sum of second attn_mask:", torch.sum(attn_mask[1]))
print("The result is 135000 because the second mask of text is 150\nThe second mask of spectrogram is 900, 150*900 = 135000")

attn_mask shape: torch.Size([2, 1000, 158])
Second attn_mask:
 tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
sum of second attn_mask: tensor(135000.)
The result is 135000 because the second mask of text is 150
The second mask of spectrogram is 900, 150*900 = 135000


In [48]:
####### Now we can implement Monotonic Alignment Search (MAS) ########
def maximum_path_each(path, value, t_y, t_x, max_neg_val=-1e9):
  index = t_x - 1

  for y in range(t_y):
    for x in range(max(0, t_x + y - t_y), min(t_x, y + 1)):
      if x == y:
        v_cur = max_neg_val
      else:
        v_cur = value[y-1][x]
      if x == 0:
        if y == 0:
          v_prev = 0.
        else:
          v_prev = max_neg_val
      else:
        v_prev = value[y-1][x-1]
      value[y][x] += max(v_prev, v_cur)

  for y in range(t_y - 1, -1, -1):
    path[y][index] = 1
    if index != 0 and (index == y or value[y-1][index] < value[y-1][index-1]):
      index = index - 1


def maximum_path_c(paths, values, t_ys, t_xs):
  b = len(paths)
  for i in range(b):
    maximum_path_each(paths[i], values[i], t_ys[i], t_xs[i])


In [49]:
neg_cent.shape

torch.Size([2, 1000, 158])

In [50]:
path = np.zeros(neg_cent.shape, dtype=np.int32)
neg_cent = neg_cent.data.numpy().astype(np.float32)
t_s_max = attn_mask.sum(1)[:, 0].data.cpu().numpy().astype(np.int32)
t_t_max = attn_mask.sum(2)[:, 0].data.cpu().numpy().astype(np.int32)
maximum_path_c(path, neg_cent, t_s_max, t_t_max)
path = torch.from_numpy(path).unsqueeze(1)

In [51]:
print("path shape:", path.shape)
print("path:\n", path[1])
print("The Right bottm corner is 0 because there is attn_mask.")
print("In index 1, the right bottm is actually path[1][899][149]:", path[1][0][899][149])
print("sum of path[0]:", path[0][0].sum())
print("sum of path[1]:", path[1][0].sum())
print("It is exact the length of spectrogram because we can not skip any spectrogram and use any frame of spectrogram twice.")

path shape: torch.Size([2, 1, 1000, 158])
path:
 tensor([[[1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.int32)
The Right bottm corner is 0 because there is attn_mask.
In index 1, the right bottm is actually path[1][899][149]: tensor(1, dtype=torch.int32)
sum of path[0]: tensor(1000)
sum of path[1]: tensor(900)
It is exact the length of spectrogram because we can not skip any spectrogram and use any frame of spectrogram twice.


In [52]:
############ Convert the path into another format ############
path = path.sum(2)

In [53]:
print(path[0])
# the value stands for the length of spectrogram each text occupy.
# For instance, if path[0][0][3] is 15
# Then it means the fourth text(actually phoneme) occupy 15 frames of spectrogram.

tensor([[  2,   1,   1,   2,   1,   1,   1,   1,   2,   1,   1,   3,   3,   6,
           9,   5,   1,   1,  10,   1,   2,   5,   1,   1,   1,   1,   2,   1,
           5,   1,   1,   2,   1,   2,   1,   2,   1,   1,   3,   1,   1,   1,
           4,   1,   2,   1,   1,   1,   1,   1,   2,   1,   1,   2,   3,   3,
           1,   3,   1,   1,   1,   6,   2,   2,   2,   2,   1,   1,   1,   4,
           1,   1,   1,   1,   3,   1,   1,   1,   1, 741,   4,   8,   1,   1,
           1,   1,   3,   1,   2,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   2,   2,   1,   3,   2,   1,   1,   1,   1,   1,   1,   1,
           1,   3,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   2,   1,   1,   1,   1,
           1,   1,   6,   1,   1,   1,   1,   1,   1,   3,   1,   1,   1,   1,
           1,   1,   1,   1]])


In [54]:
###### At Last, we comes to Duration Predictor ######
class DurationPredictor(nn.Module):
  def __init__(self, filter_channels=256, kernel_size=3, p_dropout=0.5):
    super().__init__()

    self.drop = nn.Dropout(p_dropout)
    self.conv_1 = nn.Conv1d(hidden_channels, filter_channels, kernel_size, padding=kernel_size//2)
    self.norm_1 = LayerNorm(filter_channels)
    self.conv_2 = nn.Conv1d(filter_channels, filter_channels, kernel_size, padding=kernel_size//2)
    self.norm_2 = LayerNorm(filter_channels)
    self.proj = nn.Conv1d(filter_channels, 1, 1)

  def forward(self, x, x_mask):
    x = torch.detach(x) # Stop Gradient, we don't calculate gradient here to optimize the alignment

    x = self.conv_1(x * x_mask)
    x = torch.relu(x)
    x = self.norm_1(x)
    x = self.drop(x)
    x = self.conv_2(x * x_mask)
    x = torch.relu(x)
    x = self.norm_2(x)
    x = self.drop(x)
    x = self.proj(x * x_mask)
    return x * x_mask

In [55]:
dp = DurationPredictor()
print("DurationPredictor:", dp)
print("number of parameters:", sum(param.numel() for param in dp.parameters()))

DurationPredictor: DurationPredictor(
  (drop): Dropout(p=0.5, inplace=False)
  (conv_1): Conv1d(192, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (norm_1): LayerNorm()
  (conv_2): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (norm_2): LayerNorm()
  (proj): Conv1d(256, 1, kernel_size=(1,), stride=(1,))
)
number of parameters: 345857


In [56]:
res_dp = dp(res_TextEncoder[0], res_TextEncoder[3])

In [57]:
print("dp shape:", res_dp.shape) # (batch, 1, text_len)
# Each value stands for the log length of spectrogram each text occupy.
# You may notice there are two predictor for length of spectrogram for each text
# One is the duration predictor, another is the alignment matrix.
# Duration predictor is used in Inference, and alignment matrix is used in training.

dp shape: torch.Size([2, 1, 158])


In [58]:
# We then have Discriminator, because VITS adopts Adversarial Training

In [59]:
class DiscriminatorS(torch.nn.Module):
    def __init__(self):
        super(DiscriminatorS, self).__init__()
        norm_f = weight_norm
        self.convs = nn.ModuleList([
            norm_f(Conv1d(1, 16, 15, 1, padding=7)),
            norm_f(Conv1d(16, 64, 41, 4, groups=4, padding=20)),
            norm_f(Conv1d(64, 256, 41, 4, groups=16, padding=20)),
            norm_f(Conv1d(256, 1024, 41, 4, groups=64, padding=20)),
            norm_f(Conv1d(1024, 1024, 41, 4, groups=256, padding=20)),
            norm_f(Conv1d(1024, 1024, 5, 1, padding=2)),
        ])
        self.conv_post = norm_f(Conv1d(1024, 1, 3, 1, padding=1))

    def forward(self, x):
        fmap = []

        for l in self.convs:
            x = l(x)
            x = F.leaky_relu(x, 0.1)
            fmap.append(x)
        x = self.conv_post(x)
        fmap.append(x)
        x = torch.flatten(x, 1, -1)

        return x, fmap

In [60]:
# The input of Discriminator is the waveform.
ds = DiscriminatorS()
print("DiscriminatorS:", ds)
print("number of parameters:", sum(param.numel() for param in ds.parameters()))

DiscriminatorS: DiscriminatorS(
  (convs): ModuleList(
    (0): Conv1d(1, 16, kernel_size=(15,), stride=(1,), padding=(7,))
    (1): Conv1d(16, 64, kernel_size=(41,), stride=(4,), padding=(20,), groups=4)
    (2): Conv1d(64, 256, kernel_size=(41,), stride=(4,), padding=(20,), groups=16)
    (3): Conv1d(256, 1024, kernel_size=(41,), stride=(4,), padding=(20,), groups=64)
    (4): Conv1d(1024, 1024, kernel_size=(41,), stride=(4,), padding=(20,), groups=256)
    (5): Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
  )
  (conv_post): Conv1d(1024, 1, kernel_size=(3,), stride=(1,), padding=(1,))
)
number of parameters: 5641362


In [61]:
res_DiscriminatorS = ds(res_Decoder_PosteriorEncoder)

In [69]:
print(res_DiscriminatorS[0].shape) # (batch, 1000).
print(len(res_DiscriminatorS[1])) # A list of feature maps of each layer(the output of each layers).

torch.Size([2, 1000])
7
